# Function Calling  
Permette ai modelli di recuperare dati ed eseguire azioni.  

La *chiamata di funzioni* offre un metodo potente e flessibile per far interagire i modelli OpenAI con il tuo codice o servizi esterni. Ha due principali casi d’uso:  

- **Recupero di Dati**: Consente di ottenere informazioni aggiornate da incorporare nella risposta del modello (*Retrieval-Augmented Generation - RAG*). È utile per cercare informazioni in basi di conoscenza e recuperare dati specifici da API (ad esempio, previsioni meteo attuali).  
- **Esecuzione di Azioni**: Permette di compiere operazioni come l’invio di moduli, la chiamata di API, la modifica dello stato di un’applicazione (UI/frontend o backend) o l'esecuzione di flussi di lavoro automatizzati (ad esempio, trasferire una conversazione a un altro agente).  

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) # Imposta la chiave come variabile d'ambiente per sicurezza

![Weather](./function-calling-diagram-steps.png)

In [8]:
import requests
import json

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion_2.choices[0].message.content)

The current temperature in Paris is 4.3°C.
